# NarrativeKoGPT2 학습

## 1.Google Drive 연동
- 모델 파일과 학습 데이터가 저장 되어있는 구글 드라이브의 디렉토리와 Colab을 연동.  
- 좌측상단 메뉴에서 런타임-> 런타임 유형 변경 -> 하드웨어 가속기 -> GPU 선택 후 저장

### 1.1 GPU 연동 확인

In [4]:
!nvidia-smi

Thu Dec 17 13:15:45 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 455.45.01    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 2080    Off  | 00000000:17:00.0 Off |                  N/A |
| 18%   23C    P8    11W / 215W |     10MiB /  7982MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 2080    Off  | 00000000:B3:00.0  On |                  N/A |
| 18%   

### 1.2 Google Drive 연동
아래 코드를 실행후 나오는 URL을 클릭하여 나오는 인증 코드 입력

In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [0]:
!ls

drive  sample_data


**Colab 디렉토리 아래 NarrativeKoGPT2 경로 확인**

In [4]:
!ls drive/'My Drive'/'Colab Notebooks'/

ls: 'drive/My Drive/Colab Notebooks/'에 접근할 수 없습니다: 그런 파일이나 디렉터리가 없습니다


**필요 패키지들 설치**

In [3]:
!pip install -r drive/'My Drive'/'Colab Notebooks'/NarrativeKoGPT2/requirements.txt

     |████████████████████████████████| 256kB 5.1MB/s 
     |████████████████████████████████| 68.7MB 44kB/s 
     |████████████████████████████████| 1.0MB 54.3MB/s 
     |████████████████████████████████| 501kB 51.3MB/s 
     |████████████████████████████████| 3.7MB 56.7MB/s 
     |████████████████████████████████| 870kB 57.7MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.9.1-cp36-cp36m-linux_x86_64.whl size=470974 sha256=1952a212eebb3e8949417908c0f399ab4d383f3dbc0acedd4f219c440260ba14
  Stored in directory: /root/.cache/pip/wheels/af/60/16/1f8a40e68b85bd9bd7960e91830bca5e40cd113f3220b7e231
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=4cc8105063ef093f8c730b0326be3e98f77791a6f5aec4852170b12d4bcc695c
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built gluonnlp sacremoses
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
     

In [4]:
import os

import sys
sys.path.append('drive/My Drive/Colab Notebooks/')
print(os.getcwd())


/content


## 2.KoGPT2 Transfer Learning

### 2.1.Import Package

In [10]:
import random
import torch
from torch.utils.data import DataLoader # 데이터로더
from gluonnlp.data import SentencepieceTokenizer 
from NarrativeKoGPT2.kogpt2.utils import get_tokenizer
from NarrativeKoGPT2.kogpt2.utils import download, tokenizer
from NarrativeKoGPT2.model.torch_gpt2 import GPT2Config, GPT2LMHeadModel
from NarrativeKoGPT2.util.data import NovelDataset
import gluonnlp
from tqdm import tqdm

/home/piai/anaconda3/envs/pytorch/lib/python3.7/site-packages/mxnet/optimizer/optimizer.py:167: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  Optimizer.opt_registry[name].__name__))


ModuleNotFoundError: No module named 'NarrativeKoGPT2.kogpt2'; 'NarrativeKoGPT2' is not a package

In [11]:
import random
import torch
from torch.utils.data import DataLoader # 데이터로더
from gluonnlp.data import SentencepieceTokenizer 
from kogpt2.utils import get_tokenizer
from kogpt2.utils import download, tokenizer
from model.torch_gpt2 import GPT2Config, GPT2LMHeadModel
from util.data import NovelDataset
import gluonnlp
from tqdm import tqdm
#import 위치 맞춰줌 앞의 NarrativeKoGPT2 필요없음 현재 폴더 기준이므로

In [8]:
pip install NarrativeKoGPT2

ERROR: Could not find a version that satisfies the requirement NarrativeKoGPT2
ERROR: No matching distribution found for NarrativeKoGPT2
Note: you may need to restart the kernel to use updated packages.


**torch GPU 확인**

In [12]:
print(torch.cuda.device_count())  # GPU deviec count check

2


### 2.2. koGPT-2 Config

In [35]:
ctx= 'cuda'#'cuda' #'cpu' #학습 Device CPU or GPU. colab의 경우 GPU 사용
cachedir='~/kogpt2/' # KoGPT-2 모델 다운로드 경로
epoch =30  # 학습 epoch
save_path = '/home/piai/AI project/NarrativeKoGPT2/narrativeKoGPT2/checkpoint/'
#use_cuda = True # Colab내 GPU 사용을 위한 값

pytorch_kogpt2 = {
    'url':
    'https://kobert.blob.core.windows.net/models/kogpt2/pytorch/pytorch_kogpt2_676e9bcfa7.params',
    'fname': 'pytorch_kogpt2_676e9bcfa7.params',
    'chksum': '676e9bcfa7'
}
kogpt2_config = {
    "initializer_range": 0.02,
    "layer_norm_epsilon": 1e-05,
    "n_ctx": 1024,
    "n_embd": 768,
    "n_head": 12,
    "n_layer": 12,
    "n_positions": 1024,
    "vocab_size": 50000 # 50000기본
}

### 2.3 Model and Vocab Download

In [33]:
# download model
model_info = pytorch_kogpt2
model_path = download(model_info['url'],
                       model_info['fname'],
                       model_info['chksum'],
                       cachedir=cachedir)
# download vocab
vocab_info = tokenizer
vocab_path = download(vocab_info['url'],
                       vocab_info['fname'],
                       vocab_info['chksum'],
                       cachedir=cachedir)

using cached model
using cached model


### 2.4.KoGPT-2 Model Vocab

In [36]:
# KoGPT-2 언어 모델 학습을 위한 GPT2LMHeadModel 선언
kogpt2model = GPT2LMHeadModel(config=GPT2Config.from_dict(kogpt2_config))
# model_path로부터 다운로드 받은 내용을 load_state_dict으로 업로드
kogpt2model.load_state_dict(torch.load(model_path))

device = torch.device(ctx)
kogpt2model.to(device)

# kogpt2model.eval()
# 추가로 학습하기 위해 .train() 사용
kogpt2model.train()
vocab_b_obj = gluonnlp.vocab.BERTVocab.from_sentencepiece(vocab_path,
                                                     mask_token=None,
                                                     sep_token=None,
                                                     cls_token=None,
                                                     unknown_token='<unk>',
                                                     padding_token='<pad>',
                                                     bos_token='<s>',
                                                     eos_token='</s>')


### 2.5. Get Batch Data using DataLoader

In [37]:
tok_path = get_tokenizer()
model, vocab = kogpt2model, vocab_b_obj
sentencepieceTokenizer = SentencepieceTokenizer(tok_path)

#os.chdir("../")
data_file_path = '/home/piai/AI project/Dataset/ratings_train_document_noindex_1000.csv'
batch_size = 1
novel_dataset = NovelDataset(data_file_path, vocab,sentencepieceTokenizer)
novel_data_loader = DataLoader(novel_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

using cached model


### 2.6. Learning rate, Loss function, Adam Optimizer

In [38]:
learning_rate = 1e-4
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [39]:
import subprocess

def get_gpu_memory_map():
    """Get the current gpu usage.

    Returns
    -------
    usage: dict
        Keys are device ids as integers.
        Values are memory usage as integers in MB.
    """
    result = subprocess.check_output(
        [
            'nvidia-smi', '--query-gpu=memory.used',
            '--format=csv,nounits,noheader'
        ], encoding='utf-8')
    # Convert lines into a dictionary
    gpu_memory = [int(x) for x in result.strip().split('\n')]
    gpu_memory_map = dict(zip(range(len(gpu_memory)), gpu_memory))
    return gpu_memory_map

### 2.7. KoGPT-2 Transfer Laerning

In [20]:
torch.cuda


<module 'torch.cuda' from '/home/piai/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/cuda/__init__.py'>

In [40]:
print('KoGPT-2 Transfer Learning Start')
epoch=30# 원래 200
for epoch in range(epoch):
  count = 0
  for data in novel_data_loader:
    optimizer.zero_grad()
    data = torch.stack(data) # list of Tensor로 구성되어 있기 때문에 list를 stack을 통해 변환해준다.

    data= data.transpose(1,0)
    data= data.to(ctx)
    
    outputs = model(data, labels=data)
    loss, logits = outputs[:2]
    loss.backward()
    optimizer.step()
    if count %10 ==0:
      print('epoch no.{} train no.{}  loss = {}' . format(epoch, count+1, loss))
      # torch.save(model,save_path+'checkpoint_{}_{}.tar'.format(epoch,count))
      # 추론 및 학습 재개를 위한 일반 체크포인트 저장하기
    if (count >0 and count%100==0) or (len(data) < batch_size):
      torch.save({
        'epoch': epoch,
        'train_no': count,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss':loss
      }, save_path+'narrativeKoGPT2_checkpoint.tar')

    count += 1

KoGPT-2 Transfer Learning Start
epoch no.0 train no.1  loss = 7.046463489532471
epoch no.0 train no.11  loss = 5.954883575439453
epoch no.0 train no.21  loss = 4.949443817138672
epoch no.0 train no.31  loss = 4.883050918579102
epoch no.0 train no.41  loss = 6.016555309295654
epoch no.0 train no.51  loss = 2.9170634746551514
epoch no.0 train no.61  loss = 5.77877140045166
epoch no.0 train no.71  loss = 4.958684921264648
epoch no.0 train no.81  loss = 4.843178749084473
epoch no.0 train no.91  loss = 5.756586074829102
epoch no.0 train no.101  loss = 6.17048454284668
epoch no.0 train no.111  loss = 4.237728595733643
epoch no.0 train no.121  loss = 5.130053997039795
epoch no.0 train no.131  loss = 4.233777046203613
epoch no.0 train no.141  loss = 3.838409900665283
epoch no.0 train no.151  loss = 5.796337127685547
epoch no.0 train no.161  loss = 5.366102695465088
epoch no.0 train no.171  loss = 4.242149353027344
epoch no.0 train no.181  loss = 5.539809703826904
epoch no.0 train no.191  loss 

epoch no.1 train no.631  loss = 3.3132355213165283
epoch no.1 train no.641  loss = 2.6580915451049805
epoch no.1 train no.651  loss = 4.2739410400390625
epoch no.1 train no.661  loss = 4.001410007476807
epoch no.1 train no.671  loss = 4.013136863708496
epoch no.1 train no.681  loss = 3.9678115844726562
epoch no.1 train no.691  loss = 3.7463414669036865
epoch no.1 train no.701  loss = 4.110046863555908
epoch no.1 train no.711  loss = 2.8329477310180664
epoch no.1 train no.721  loss = 3.436633586883545
epoch no.1 train no.731  loss = 3.5188050270080566
epoch no.1 train no.741  loss = 4.182027339935303
epoch no.1 train no.751  loss = 3.8337669372558594
epoch no.1 train no.761  loss = 4.339745998382568
epoch no.1 train no.771  loss = 4.331291198730469
epoch no.1 train no.781  loss = 3.0040550231933594
epoch no.1 train no.791  loss = 3.496892213821411
epoch no.1 train no.801  loss = 2.747851610183716
epoch no.1 train no.811  loss = 2.708343029022217
epoch no.1 train no.821  loss = 4.8098688

epoch no.3 train no.241  loss = 2.503761053085327
epoch no.3 train no.251  loss = 1.9396915435791016
epoch no.3 train no.261  loss = 3.450258255004883
epoch no.3 train no.271  loss = 2.491464376449585
epoch no.3 train no.281  loss = 2.050429105758667
epoch no.3 train no.291  loss = 1.6706172227859497
epoch no.3 train no.301  loss = 1.8905718326568604
epoch no.3 train no.311  loss = 2.6311194896698
epoch no.3 train no.321  loss = 2.465738296508789
epoch no.3 train no.331  loss = 2.7076900005340576
epoch no.3 train no.341  loss = 1.334758996963501
epoch no.3 train no.351  loss = 2.721086025238037
epoch no.3 train no.361  loss = 2.530245065689087
epoch no.3 train no.371  loss = 2.2533369064331055
epoch no.3 train no.381  loss = 2.424034595489502
epoch no.3 train no.391  loss = 1.7883328199386597
epoch no.3 train no.401  loss = 2.324291229248047
epoch no.3 train no.411  loss = 1.9557912349700928
epoch no.3 train no.421  loss = 3.0204977989196777
epoch no.3 train no.431  loss = 2.6261096000

epoch no.4 train no.861  loss = 1.3994948863983154
epoch no.4 train no.871  loss = 1.7780930995941162
epoch no.4 train no.881  loss = 2.1215174198150635
epoch no.4 train no.891  loss = 1.953765869140625
epoch no.4 train no.901  loss = 1.9037717580795288
epoch no.4 train no.911  loss = 2.0179367065429688
epoch no.4 train no.921  loss = 1.5265127420425415
epoch no.4 train no.931  loss = 1.5942691564559937
epoch no.4 train no.941  loss = 1.2496191263198853
epoch no.4 train no.951  loss = 1.5432502031326294
epoch no.4 train no.961  loss = 1.8202329874038696
epoch no.4 train no.971  loss = 1.2416030168533325
epoch no.4 train no.981  loss = 1.5664693117141724
epoch no.4 train no.991  loss = 1.917365312576294
epoch no.4 train no.1001  loss = 2.254119634628296
epoch no.5 train no.1  loss = 1.3908140659332275
epoch no.5 train no.11  loss = 1.6050950288772583
epoch no.5 train no.21  loss = 1.163466215133667
epoch no.5 train no.31  loss = 0.7416223883628845
epoch no.5 train no.41  loss = 1.037582

epoch no.6 train no.461  loss = 0.9893984198570251
epoch no.6 train no.471  loss = 1.320799469947815
epoch no.6 train no.481  loss = 1.7558101415634155
epoch no.6 train no.491  loss = 1.0248117446899414
epoch no.6 train no.501  loss = 1.4232122898101807
epoch no.6 train no.511  loss = 0.769058346748352
epoch no.6 train no.521  loss = 1.6908988952636719
epoch no.6 train no.531  loss = 1.8844220638275146
epoch no.6 train no.541  loss = 1.1760354042053223
epoch no.6 train no.551  loss = 2.1328938007354736
epoch no.6 train no.561  loss = 2.4283065795898438
epoch no.6 train no.571  loss = 1.673911213874817
epoch no.6 train no.581  loss = 0.6497252583503723
epoch no.6 train no.591  loss = 0.8961417078971863
epoch no.6 train no.601  loss = 0.8632442355155945
epoch no.6 train no.611  loss = 1.830298900604248
epoch no.6 train no.621  loss = 2.5848562717437744
epoch no.6 train no.631  loss = 1.4887547492980957
epoch no.6 train no.641  loss = 1.0304055213928223
epoch no.6 train no.651  loss = 0.9

...

epoch no.22 train no.191  loss = 0.5682371258735657
epoch no.22 train no.201  loss = 0.2681303322315216
epoch no.22 train no.211  loss = 0.2923632860183716
epoch no.22 train no.221  loss = 1.280716061592102
epoch no.22 train no.231  loss = 0.44784268736839294
epoch no.22 train no.241  loss = 0.28963345289230347
epoch no.22 train no.251  loss = 0.5053640604019165
epoch no.22 train no.261  loss = 0.6291272044181824
epoch no.22 train no.271  loss = 0.29244184494018555
epoch no.22 train no.281  loss = 0.48942461609840393
epoch no.22 train no.291  loss = 0.32050150632858276
epoch no.22 train no.301  loss = 0.38151776790618896
epoch no.22 train no.311  loss = 0.17940975725650787
epoch no.22 train no.321  loss = 0.828920304775238
epoch no.22 train no.331  loss = 0.6562354564666748
epoch no.22 train no.341  loss = 0.2680119574069977
epoch no.22 train no.351  loss = 0.4543781280517578
epoch no.22 train no.361  loss = 0.5639548897743225
epoch no.22 train no.371  loss = 0.3891828954219818
epoch n

epoch no.23 train no.761  loss = 0.33282002806663513
epoch no.23 train no.771  loss = 0.2797936797142029
epoch no.23 train no.781  loss = 0.46506500244140625
epoch no.23 train no.791  loss = 0.7814712524414062
epoch no.23 train no.801  loss = 0.7744285464286804
epoch no.23 train no.811  loss = 0.788564145565033
epoch no.23 train no.821  loss = 0.21603751182556152
epoch no.23 train no.831  loss = 0.8350812792778015
epoch no.23 train no.841  loss = 0.4251910150051117
epoch no.23 train no.851  loss = 0.45540785789489746
epoch no.23 train no.861  loss = 0.9145691990852356
epoch no.23 train no.871  loss = 0.31611984968185425
epoch no.23 train no.881  loss = 0.5176998972892761
epoch no.23 train no.891  loss = 0.2609080970287323
epoch no.23 train no.901  loss = 0.29188039898872375
epoch no.23 train no.911  loss = 0.28609031438827515
epoch no.23 train no.921  loss = 0.22093184292316437
epoch no.23 train no.931  loss = 0.39562851190567017
epoch no.23 train no.941  loss = 0.3391624391078949
epoc

epoch no.25 train no.321  loss = 1.00715970993042
epoch no.25 train no.331  loss = 0.9148460030555725
epoch no.25 train no.341  loss = 0.7659007906913757
epoch no.25 train no.351  loss = 0.29396891593933105
epoch no.25 train no.361  loss = 0.5903295874595642
epoch no.25 train no.371  loss = 0.2950918972492218
epoch no.25 train no.381  loss = 0.5587780475616455
epoch no.25 train no.391  loss = 1.7822535037994385
epoch no.25 train no.401  loss = 0.408791720867157
epoch no.25 train no.411  loss = 0.6272550225257874
epoch no.25 train no.421  loss = 0.43762996792793274
epoch no.25 train no.431  loss = 0.21939566731452942
epoch no.25 train no.441  loss = 0.41066446900367737
epoch no.25 train no.451  loss = 0.5636815428733826
epoch no.25 train no.461  loss = 0.6404642462730408
epoch no.25 train no.471  loss = 0.5448951125144958
epoch no.25 train no.481  loss = 0.38892441987991333
epoch no.25 train no.491  loss = 0.32923316955566406
epoch no.25 train no.501  loss = 0.3018069565296173
epoch no.

epoch no.26 train no.891  loss = 0.7115361094474792
epoch no.26 train no.901  loss = 0.3137188255786896
epoch no.26 train no.911  loss = 0.5961471199989319
epoch no.26 train no.921  loss = 0.8618901371955872
epoch no.26 train no.931  loss = 0.6378262639045715
epoch no.26 train no.941  loss = 0.40626904368400574
epoch no.26 train no.951  loss = 0.580567479133606
epoch no.26 train no.961  loss = 0.42605435848236084
epoch no.26 train no.971  loss = 0.6921471953392029
epoch no.26 train no.981  loss = 0.8511694669723511
epoch no.26 train no.991  loss = 0.6347464919090271
epoch no.26 train no.1001  loss = 0.191537544131279
epoch no.27 train no.1  loss = 0.18753103911876678
epoch no.27 train no.11  loss = 0.7883486747741699
epoch no.27 train no.21  loss = 0.26311057806015015
epoch no.27 train no.31  loss = 0.5828409790992737
epoch no.27 train no.41  loss = 0.34285035729408264
epoch no.27 train no.51  loss = 0.4306698143482208
epoch no.27 train no.61  loss = 0.670984148979187
epoch no.27 train

epoch no.28 train no.451  loss = 0.30101487040519714
epoch no.28 train no.461  loss = 0.8013982772827148
epoch no.28 train no.471  loss = 0.3423989415168762
epoch no.28 train no.481  loss = 1.038887619972229
epoch no.28 train no.491  loss = 0.31040525436401367
epoch no.28 train no.501  loss = 0.4136413335800171
epoch no.28 train no.511  loss = 0.40437591075897217
epoch no.28 train no.521  loss = 0.3950178027153015
epoch no.28 train no.531  loss = 0.8377379179000854
epoch no.28 train no.541  loss = 0.8162409663200378
epoch no.28 train no.551  loss = 0.44748422503471375
epoch no.28 train no.561  loss = 0.27086150646209717
epoch no.28 train no.571  loss = 0.6789461970329285
epoch no.28 train no.581  loss = 0.7552570104598999
epoch no.28 train no.591  loss = 0.8153665661811829
epoch no.28 train no.601  loss = 0.4967116415500641
epoch no.28 train no.611  loss = 0.28846657276153564
epoch no.28 train no.621  loss = 0.2595047652721405
epoch no.28 train no.631  loss = 0.9938064217567444
epoch n